# Histogram and Intensity Transformations¶

Pixel Transforms are operations you perform one pixel at a time. In this lab, you will start by creating histograms. Histograms display the intensity of the image and can be used to optimize image characteristics. You will then apply Intensity Transformations, making objects easier to see by improving image contrast and brightness. In the last portion of the lab, you will use thresholding to segment objects from images.
<br>
Pixel Transforms<br>
Histograms<br>
Intensity Transformations<br>
Thresholding and Simple Segmentation

In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
def plot_image(image_1, image_2, title_1 = "Original", title_2 = "new Image"):
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.imshow(image_1, cmap = 'gray')
    plt.title()